In [3]:
import cv2
import time
import os
from skimage.color import rgb2gray
from skimage.filters import sobel
import matplotlib.pyplot as plt
import numpy as np
from skimage import img_as_float
import scipy

file_count = 0

def redefine_type_dict():
    d = dict()
    use_type = [1,2,3,4,6,7,8,13,14,15,16,21,23,27,28,29,30,31,32]
    for i,t in enumerate(use_type):
        d[t] = i
    d[70] = 6
    d[80] = 7
    return d

def print_frame_list(frame):
    for f in range(frame.shape[2]):
        plt.imshow(frame[:,:,f])
        plt.show()

def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    type_dict = redefine_type_dict()
    name, _ = os.path.splitext(os.path.basename(input_loc))
    subject = name.split('_')[0]
    t = int(name.split('_')[1])
    if t not in type_dict:
        return
    else:
        t = type_dict[t]
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
#     print ("Number of frames: ", video_length)
    
    frame_list = []
    count = 0
#     print ("Converting video..")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        
        # down sample by 2
        resized_frame = cv2.resize(frame, (125,57))
        
        #convert to skimage
        resized_frame = resized_frame[:,:,::-1]
        resized_frame = img_as_float(resized_frame)
        
        # convert to gray image
        gray_image = rgb2gray(resized_frame)
        # extract sobel gradient
        edges = sobel(gray_image)

        frame_list.append(edges)
        # Write the results back to output location.
#         cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), laplacian)
        count = count + 1
        # If there are no more frames left
        if (count == video_length):
            # Log the time again
            time_end = time.time()
            cap.release()
            # stack to third dimension
            sample = np.stack(frame_list, axis=2)
            # Resample to 32 frames
            resample = scipy.ndimage.zoom(sample, (1,1,32/video_length))
            if resample.shape[2] != 32:
                print(name)
                
            global file_count
            np.save(output_loc+str(file_count)+'_'+str(t)+'_'+str(subject)+'.npy', resample)
            file_count += 1
            # Print stats
#             print (name, "Done extracting frames.\n%d frames extracted" % count)
#             print ("It took %f seconds forconversion.\n" % (time_end-time_start))
            break

In [4]:
for v in sorted(os.listdir('LISA_HG_Data/dynamic_gestures/data')):
    if v.endswith('avi'):
        video_to_frames('LISA_HG_Data/dynamic_gestures/data/'+v, 'HRN_viva_gesture_data/')
        

/Users/liumy/anaconda/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
